In [4]:
import os
import shutil
import sqlite3
import xml.etree.ElementTree as ET
import re
import scipy.stats as stats
import matplotlib.pyplot as plt
from docx import Document
from matplotlib.backends.backend_pdf import PdfPages
import datetime
import shutil
import tkinter as tk
from tkinter import filedialog
from docx.shared import Pt, Inches
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT


def select_folder():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    folder_path = filedialog.askdirectory(initialdir=r"H:\Pitching\Data", title="Select Folder")
    if folder_path:
        copy_and_move_files(folder_path)


def extract_initials(folder_name):
    initials = ''.join(filter(str.isupper, folder_name))
    return initials[::-1]  # Reverse the initials


def copy_and_move_files(base_path):
    # Get the base folder name
    base_folder = os.path.basename(base_path)

    # Get list of folders in the base path
    folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]
    # Sort folders by creation time
    folders.sort(key=lambda x: os.path.getctime(os.path.join(base_path, x)), reverse=True)

    if not folders:
        print("No folders found in the specified directory.")
        return

    latest_folder = folders[0]
    folder_path = os.path.join(base_path, latest_folder)

    # Get creation time of the latest folder
    creation_time = os.path.getctime(folder_path)
    creation_date = datetime.datetime.fromtimestamp(creation_time).strftime('%m-%d-%y')

    # Find the files to copy and rename
    files_to_copy_rename = []
    for file in os.listdir(folder_path):
        if file.lower() == 'session.xml' or file.lower() == 'session_data.xml':
            files_to_copy_rename.append(file)

    if not files_to_copy_rename:
        print("No XML files named 'session.xml' or 'session_data.xml' found in the latest folder.")
        return

    # Construct new file names
    new_file_names = []
    initials = extract_initials(base_folder)
    for file in files_to_copy_rename:
        if file.lower() == 'session.xml':
            new_file_name = f"session_{initials}_{creation_date}.xml"
        elif file.lower() == 'session_data.xml':
            new_file_name = f"session_data_{initials}_{creation_date}.xml"
        else:
            continue
        new_file_names.append(new_file_name)

    # Create temporary directory to store renamed copies
    temp_dir = os.path.join(base_path, "temp_rename")
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)

    # Copy and rename the files to the temporary directory
    for original_file, new_file_name in zip(files_to_copy_rename, new_file_names):
        original_file_path = os.path.join(folder_path, original_file)
        new_file_path = os.path.join(temp_dir, new_file_name)
        shutil.copyfile(original_file_path, new_file_path)
        print(f"File '{original_file}' copied and renamed to: {new_file_path}")

    # Copy the renamed files to the destination folder
    new_folder_path = r"F:\My Drive\Pitching Data\New Data"
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)
    for new_file_name in new_file_names:
        temp_file_path = os.path.join(temp_dir, new_file_name)
        destination_file_path = os.path.join(new_folder_path, new_file_name)
        shutil.copyfile(temp_file_path, destination_file_path)
        print(f"File '{new_file_name}' copied to: {destination_file_path}")
        
# Prompt user to select the folder
select_folder()


# Source and destination directory paths
source_dir = r"F:\My Drive\Pitching Data\New Data"
destination_dir = r"C:\Users\q\PycharmProjects\Rename_Session.xml\Pitching Data\New Data"

# Iterate over files in the source directory
for file_name in os.listdir(source_dir):
    if file_name.endswith('.xml'):
        source_file_path = os.path.join(source_dir, file_name)
        destination_file_path = os.path.join(destination_dir, file_name)
        shutil.move(source_file_path, destination_file_path)
        print(f"File '{file_name}' moved to: {destination_file_path}")

print("XML files copied successfully!")

File 'session_data.xml' copied and renamed to: H:/Pitching/Data/Weiss, Ryan 03-22\temp_rename\session_data_RW_10-20-22.xml
File 'session_data_RW_10-20-22.xml' copied to: F:\My Drive\Pitching Data\New Data\session_data_RW_10-20-22.xml
File 'session_RW_10-20-22.xml' moved to: C:\Users\q\PycharmProjects\Rename_Session.xml\Pitching Data\New Data\session_RW_10-20-22.xml
File 'session_data_RW_10-20-22.xml' moved to: C:\Users\q\PycharmProjects\Rename_Session.xml\Pitching Data\New Data\session_data_RW_10-20-22.xml
XML files copied successfully!
